## Thread pool
---
執行緒池是一種用於管理執行緒的機制，它可以維護一組執行緒，並在需要時分配執行緒來執行任務。執行緒池可以減少執行緒的創建和銷毀次數，提高執行緒的重用率，從而提高系統的性能。

為何需要Thread pool(與Semaphore的差異)
  + 主線程中可以獲取某一個線程的狀態或者某一個執行緒的返回值。
  + 當一個線程完成時，主線程可以立即知道。
  + futures包可以讓多線程和多進程編程接口一致，要做切換會非常容易。


In [15]:
import concurrent.futures
import threading
import time

# 網頁爬蟲類別 (目的:控制同時工作爬蟲的數量，避免被封IP)
class HtmlSpider:
    def __init__(self, name, lock):
        self.name = name
        self.lock = lock

    def fetch_html(self):
        time.sleep(2)
        with self.lock:
            print(f"{self.name} got html text success in ", time.strftime("%H:%M:%S", time.localtime()), flush=True)
        return self.name+" done"

# 獲取網址內容類別
class UrlProducer:
    def __init__(self, max_workers):
        self.lock = threading.Lock()
        self.max_workers = max_workers

    def produce_urls(self):
        print("UrlProducer run produce_urls()", flush=True)
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            futures = [executor.submit(HtmlSpider(f"htmlSpider-{i}", self.lock).fetch_html) for i in range(20)]
            # 若有任務完成的，將它的結果印出來
            for future in concurrent.futures.as_completed(futures):
                with self.lock:
                  print(future.result())
            concurrent.futures.wait(futures) # 等待所有HtmlSpider執行緒結束
            print("All threads are done")

urlProducer = UrlProducer(max_workers=3)
urlProducer.produce_urls()

UrlProducer run produce_urls()
htmlSpider-1 got html text success in  18:49:44
htmlSpider-0 got html text success in  18:49:44
htmlSpider-2 got html text success in  18:49:44
htmlSpider-1 done
htmlSpider-0 done
htmlSpider-2 done
htmlSpider-3 got html text success in  18:49:46
htmlSpider-3 done
htmlSpider-4 got html text success in  18:49:46
htmlSpider-4 done
htmlSpider-5 got html text success in  18:49:46
htmlSpider-5 done
htmlSpider-6 got html text success in  18:49:48
htmlSpider-6 done
htmlSpider-7 got html text success in  18:49:48
htmlSpider-7 done
htmlSpider-8 got html text success in  18:49:48
htmlSpider-8 done
htmlSpider-9 got html text success in  18:49:50
htmlSpider-9 done
htmlSpider-10 got html text success in  18:49:50
htmlSpider-10 done
htmlSpider-11 got html text success in  18:49:50
htmlSpider-11 done
htmlSpider-12 got html text success in  18:49:52
htmlSpider-12 done
htmlSpider-13 got html text success in  18:49:52
htmlSpider-13 done
htmlSpider-14 got html text success i